In [1]:
import pandas as pd
import re
import numpy as np
import emosent
from emosent import get_emoji_sentiment_rank_multiple
from emosent import get_emoji_sentiment_rank

import emoji
import re

In [2]:
df = pd.read_csv('/workspaces/Proyecto_Final_Prueba/data/df_filtrado_solo75.csv')

In [3]:
def categorize_expressions(tweet):
    question_pattern = r'\?'
    exclamation_pattern = r'!'
    laugh_pattern = r'\b(?:ha|he|ja|je|hi){2,}\b' 
   

    question = bool(re.search(question_pattern, tweet))
    exclamation = bool(re.search(exclamation_pattern, tweet))
    laugh = bool(re.search(laugh_pattern, tweet, re.IGNORECASE))  

    return question, exclamation, laugh

def process_tweets(df):
    results = df['tweet'].apply(categorize_expressions)
    df['Interrogación'], df['Exclamación'], df['Risa'] = zip(*results)
    df['Interrogación'] = df['Interrogación'].astype(int)
    df['Exclamación'] = df['Exclamación'].astype(int)
    df['Risa'] = df['Risa'].astype(int)
    return df

In [54]:
process_tweets(df)

,class,tweet,user_id,salida_diagnostica,Interrogación,Exclamación,Risa,Emojis_detectados,Sentiment_score,Emojis_Sentimentscore,Emojis_filtrados,Sentiment_score_filtrado,Numero_emojis_por_tweet,Uso_de_emojis
0,ADHD,"""@USER AAABDVSGJS NOO you're too kind 😭🥺 more ...",1276497310667673604,diagnosticado,0,0,0,"[😭, 🥺, 😤, 🥺, 🥰, 💞, 💕]","[-0.093, None, -0.211, None, None, 0.742, 0.633]","{(🥺, None), (😤, -0.211), (🥰, None), (😭, -0.093...","[😤, 😭, 💞, 💕]","[-0.093, -0.211, 0.742, 0.633]",7,1
1,ADHD,"""wow! 🤩 two years here HTTPURL""",1276497310667673604,diagnosticado,0,1,0,[🤩],[None],"{(🤩, None)}",[],[],1,1
2,ADHD,"""nothing can compare to RK900 manboobies thoug...",1276497310667673604,diagnosticado,0,0,0,[😩],[-0.368],"{(😩, -0.368)}",[😩],[-0.368],1,1
3,ADHD,"""too bad his titties weren't bara sized huge a...",1276497310667673604,diagnosticado,0,0,0,"[😩, 🎂]","[-0.368, 0.622]","{(😩, -0.368), (🎂, 0.622)}","[😩, 🎂]","[-0.368, 0.622]",2,1
4,ADHD,"""ended up paying more than what was initially ...",1276497310667673604,diagnosticado,0,0,0,[😪],[-0.081],"{(😪, -0.081)}",[😪],[-0.081],1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3025882,SCHIZOPHRENIA,"""""They all spoke powerfully about psychiatry's...",2377367690,diagnosticado,0,0,0,[],[],{},[],[],0,0
3025883,SCHIZOPHRENIA,"""@USER Thank you!""",2377367690,diagnosticado,0,1,0,[],[],{},[],[],0,0
3025884,SCHIZOPHRENIA,"""Our Movies of the Mind column continues with ...",2377367690,diagnosticado,0,0,0,[],[],{},[],[],0,0
3025885,SCHIZOPHRENIA,"""Sigmund Freuds article ""shows a meticulous cl...",2377367690,diagnosticado,1,0,0,[],[],{},[],[],0,0


In [55]:
print(df['Risa'].sum())
print(df['Interrogación'].sum())
print(df['Exclamación'].sum())

13567
314243
482608


In [56]:
print(df[df['salida_diagnostica'] == 'diagnosticado']['Risa'].sum())
print(df[df['salida_diagnostica'] == 'diagnosticado']['Interrogación'].sum())
print(df[df['salida_diagnostica'] == 'diagnosticado']['Exclamación'].sum())

8354
151317
231938


## Emojis

In [5]:
# Función para detectar el total de emojis en un tweet
def total_emojis(tweet):
    total_emojis_detectados = []
    
   
    for char in tweet:
        if emoji.emoji_count(char):
            total_emojis_detectados.append(char)
    
    return total_emojis_detectados

df['Emojis_detectados'] = df['tweet'].apply(total_emojis)

In [8]:
# Función que saca el sentiment_score del total de emojis encontrados
def get_sentiment_score(emoji):
    try:
        emoji_rank = get_emoji_sentiment_rank(emoji)
        if emoji_rank is not None:
            sentiment_score = emoji_rank.get('sentiment_score')
        else:
            sentiment_score = None
    except KeyError:
        # Si el emoji no tiene una puntuación de sentimiento, se asigna como NaN
        sentiment_score = None
    return sentiment_score


df['Sentiment_score'] = df['Emojis_detectados'].apply(lambda emojis: [get_sentiment_score(emoji) for emoji in emojis])

In [10]:
def detectar_emojis_completo(tweet):
    emojis_detectados = []
    
    # Iterar sobre cada carácter en el tweet
    for char in tweet:
        # Verificar si el carácter es un emoji
        if emoji.is_emoji(char):
            # Obtener el valor de sentiment_score del emoji
            emoji_sentiment_rank = get_emoji_sentiment_rank(char)
            if emoji_sentiment_rank:
                sentiment_score = emoji_sentiment_rank['sentiment_score']
                emojis_detectados.append((char, sentiment_score))
            else:
                # Si el emoji no tiene sentiment_score, agregarlo con None
                emojis_detectados.append((char, None))
    
    return emojis_detectados

df['Emojis_Sentimentscore'] = df['tweet'].apply(detectar_emojis_completo)

In [16]:
emojis_con_valor_none = []
# Iterar sobre cada fila de la columna
for row in df['Emojis_Sentimentscore']:
    # Iterar sobre cada tupla de emoji y su valor de sentimiento en la fila
    for emoji, sentiment_score in row:
        # Verificar si el valor de sentimiento es None
        if sentiment_score is None:
            # Agregar el emoji a la lista si su valor de sentimiento es None
            emojis_con_valor_none.append(emoji)

# Convertir la lista a un conjunto para eliminar duplicados
emojis_con_valor_none = set(emojis_con_valor_none)

print("Emojis con valor None:", emojis_con_valor_none)

Emojis con valor None: {'🟦', '🥚', '🤎', '🪀', '🤓', '🍾', '🦁', '🥭', '🧧', '↘', '🥯', '⁉', '🦎', '🥦', '🥢', '🗺', '🌪', '🦨', '🕸', '☸', '🥖', '🏋', '🥶', '🕹', '🟪', '🛺', '🦛', '🥵', '🪘', '🙁', '🫑', '🥅', '🦥', '🧂', '🥺', '⏸', '🪰', '🥛', '🏏', '🪒', '⚛', '🟡', '🧮', '🦀', '🦜', '\U0001faf5', '🤷', '🕍', '🕺', '🤤', '🪤', '⚧', '♒', '🥙', '🤟', '🤛', '🥉', '🕗', '🗄', '🤡', '🪖', '🤪', '🏗', '📸', '🥳', '🦍', '🦸', '🥬', '🧩', '📩', '🦫', '🥾', '🛎', '🥁', '🧱', '🩳', '☦', '🧊', '🛅', '\U0001f9cc', '◾', '🫓', '\U0001fad8', '⛴', '🥌', '🤸', '🤫', '🚟', '🥫', '🥣', '🧥', '🦬', '\U0001fab8', '🧘', '📿', '🤌', '🩰', '🧰', '🪳', '\U0001fae7', '↖', '🏅', '🧽', '🐿', '⏹', '🧟', '🦒', '🤯', '🥘', '📽', '🥲', '🤼', '⛩', '🚰', '\U0001fae1', '\U0001faac', '🤾', '\U0001faf1', '🕷', '🈵', '\U0001fad7', '⚖', '🥓', '🧙', '🕉', '🪃', '🦺', '🚏', '◽', '🖕', '\U0001faa9', '🖐', '🛁', '🖨', '🤲', '🔢', '🏕', '🛋', '🤝', '㊗', '🧶', '🏜', '🏔', '🪜', '🪐', '🪢', '🟧', '🚱', '🧭', '♉', '🏖', '🌤', '🚈', '⛸', '🤏', '🌧', '🧆', '🕠', '🤱', '🔂', '🛗', '🛢', '⏫', '🦈', '🏽', '◻', '🤘', '🧓', '🛏', '\U0001faf0', '🤹', '🤶', '🧏', '⏱', '🫕', '\

In [40]:
def filtrar_emojis(emojis_lista, emojis_a_eliminar):
    return [emoji for emoji, _ in emojis_lista if emoji not in emojis_a_eliminar]

# Aplicar la función de filtrado para cada fila y crear la nueva columna
df['Emojis_filtrados'] = df['Emojis_Sentimentscore'].apply(lambda x: filtrar_emojis(x, emojis_con_valor_none))

In [43]:
def extract_emoji_sentiment_scores(tweet):
    # Obtener los emojis y sus puntajes de sentimiento del tweet
    emoji_sentiment_data = get_emoji_sentiment_rank_multiple(tweet)
    
    # Inicializar una lista para almacenar los puntajes de sentimiento de los emojis
    emoji_sentiment_scores = []
    
    # Extraer los puntajes de sentimiento de los emojis y agregarlos a la lista
    for emoji_data in emoji_sentiment_data:
        emoji_score = emoji_data['emoji_sentiment_rank']['sentiment_score']
        emoji_sentiment_scores.append(emoji_score)
    
    return emoji_sentiment_scores

# Aplicar la función a la columna de tweets
df['Sentiment_score_filtrado'] = df['tweet'].apply(extract_emoji_sentiment_scores)

In [47]:
df['Numero_emojis_por_tweet'] = df['Emojis_detectados'].apply(len)

In [50]:
df['Uso_de_emojis'] = df['Emojis_detectados'].apply(lambda x: 1 if x else 0)

In [58]:
df.head()

,class,tweet,user_id,salida_diagnostica,Interrogación,Exclamación,Risa,Emojis_detectados,Sentiment_score,Emojis_Sentimentscore,Emojis_filtrados,Sentiment_score_filtrado,Numero_emojis_por_tweet,Uso_de_emojis
0,ADHD,"""@USER AAABDVSGJS NOO you're too kind 😭🥺 more ...",1276497310667673604,diagnosticado,0,0,0,"[😭, 🥺, 😤, 🥺, 🥰, 💞, 💕]","[-0.093, None, -0.211, None, None, 0.742, 0.633]","{(🥺, None), (😤, -0.211), (🥰, None), (😭, -0.093...","[😤, 😭, 💞, 💕]","[-0.093, -0.211, 0.742, 0.633]",7,1
1,ADHD,"""wow! 🤩 two years here HTTPURL""",1276497310667673604,diagnosticado,0,1,0,[🤩],[None],"{(🤩, None)}",[],[],1,1
2,ADHD,"""nothing can compare to RK900 manboobies thoug...",1276497310667673604,diagnosticado,0,0,0,[😩],[-0.368],"{(😩, -0.368)}",[😩],[-0.368],1,1
3,ADHD,"""too bad his titties weren't bara sized huge a...",1276497310667673604,diagnosticado,0,0,0,"[😩, 🎂]","[-0.368, 0.622]","{(😩, -0.368), (🎂, 0.622)}","[😩, 🎂]","[-0.368, 0.622]",2,1
4,ADHD,"""ended up paying more than what was initially ...",1276497310667673604,diagnosticado,0,0,0,[😪],[-0.081],"{(😪, -0.081)}",[😪],[-0.081],1,1


In [ ]:
'''
emojis_con_valor_none = []
# Iterar sobre cada fila de la columna
for row in df['Emojis_Sentimentscore']:
    # Iterar sobre cada tupla de emoji y su valor de sentimiento en la fila
    for emoji, sentiment_score in row:
        # Verificar si el valor de sentimiento es None
        if sentiment_score is None:
            # Agregar el emoji a la lista si su valor de sentimiento es None
            emojis_con_valor_none.append(emoji)

# Convertir la lista a un conjunto para eliminar duplicados
emojis_con_valor_none = set(emojis_con_valor_none)

print("Emojis con valor None:", emojis_con_valor_none)
'''

Emojis con valor None: {'🟦', '🥚', '🤎', '🪀', '🤓', '🍾', '🦁', '🥭', '🧧', '↘', '🥯', '⁉', '🦎', '🥦', '🥢', '🗺', '🌪', '🦨', '🕸', '☸', '🥖', '🏋', '🥶', '🕹', '🟪', '🛺', '🦛', '🥵', '🪘', '🙁', '🫑', '🥅', '🦥', '🧂', '🥺', '⏸', '🪰', '🥛', '🏏', '🪒', '⚛', '🟡', '🧮', '🦀', '🦜', '\U0001faf5', '🤷', '🕍', '🕺', '🤤', '🪤', '⚧', '♒', '🥙', '🤟', '🤛', '🥉', '🕗', '🗄', '🤡', '🪖', '🤪', '🏗', '📸', '🥳', '🦍', '🦸', '🥬', '🧩', '📩', '🦫', '🥾', '🛎', '🥁', '🧱', '🩳', '☦', '🧊', '🛅', '\U0001f9cc', '◾', '🫓', '\U0001fad8', '⛴', '🥌', '🤸', '🤫', '🚟', '🥫', '🥣', '🧥', '🦬', '\U0001fab8', '🧘', '📿', '🤌', '🩰', '🧰', '🪳', '\U0001fae7', '↖', '🏅', '🧽', '🐿', '⏹', '🧟', '🦒', '🤯', '🥘', '📽', '🥲', '🤼', '⛩', '🚰', '\U0001fae1', '\U0001faac', '🤾', '\U0001faf1', '🕷', '🈵', '\U0001fad7', '⚖', '🥓', '🧙', '🕉', '🪃', '🦺', '🚏', '◽', '🖕', '\U0001faa9', '🖐', '🛁', '🖨', '🤲', '🔢', '🏕', '🛋', '🤝', '㊗', '🧶', '🏜', '🏔', '🪜', '🪐', '🪢', '🟧', '🚱', '🧭', '♉', '🏖', '🌤', '🚈', '⛸', '🤏', '🌧', '🧆', '🕠', '🤱', '🔂', '🛗', '🛢', '⏫', '🦈', '🏽', '◻', '🤘', '🧓', '🛏', '\U0001faf0', '🤹', '🤶', '🧏', '⏱', '🫕', '\

In [44]:
# Contar el número de filas sin emojis
num_rows_without_emojis = (df['Emojis_filtrados'].str.len() >= 1).sum()

print("Número de filas sin emojis:", num_rows_without_emojis)

print(df.shape)

Número de filas sin emojis: 427384
(3025887, 12)


In [ ]:
# Cargar el archivo CSV que contiene la relación entre emojis y etiquetas
df_emojis = pd.read_csv("/workspaces/Proyecto_Final_Prueba/data/emojis.csv") 
df_emojis['code'] = df_emojis['code'].str.strip(':')
df_emojis

,code,Shape
0,puño_hacia_la_derecha,🤜
1,bandera_islas_marianas_del_norte,🇲🇵
2,géminis,♊
3,chef_mujer_tono_de_piel_claro_medio,👩🏼‍🍳
4,mujer_con_hiyab_tono_de_piel_oscuro,🧕🏿
...,...,...
2372,mano_haciendo_el_gesto_de_llamar_tono_de_piel_claro,🤙🏻
2373,hombre_tono_de_piel_oscuro,👨🏿
2374,cara_de_dragón,🐲
2375,hada_tono_de_piel_oscuro,🧚🏿


In [ ]:

# Función para asignar etiquetas a emojis en un tweet
def asignar_etiquetas_a_emojis(emojis_detectados, df_emojis):
    etiquetas_asignadas = []
    for emoji in emojis_detectados:
        # Buscar la etiqueta correspondiente al emoji en el DataFrame de etiquetas
        etiqueta = df_emojis[df_emojis['Shape'] == emoji]['code'].values
        # Si se encuentra la etiqueta, agregarla a la lista de etiquetas asignadas
        if len(etiqueta) > 0:
            etiquetas_asignadas.append(etiqueta[0])
        else:
            etiquetas_asignadas.append(None)  # Si no se encuentra la etiqueta, agregar None
    return etiquetas_asignadas

# Función para detectar emojis en cada tweet de la columna 'tweet' del DataFrame y asignar etiquetas
def detectar_emojis_y_asignar_etiquetas(df):
    # Aplicar la función detectar_emojis_en_tweet a cada tweet en la columna 'tweet'
    df['emojis_detectados'] = df['tweet'].apply(detectar_emojis_en_tweet)
    
    # Aplicar la función asignar_etiquetas_a_emojis a la columna 'emojis_detectados' del DataFrame
    df['etiquetas_asignadas'] = df['emojis_detectados'].apply(lambda emojis: asignar_etiquetas_a_emojis(emojis, df_emojis))
    
    return df
